In [1]:
from grobid.client import GrobidClient
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import xml.etree.ElementTree as ET
import os

<h1>Modelo para tokenizar</h1>

In [2]:
model = SentenceTransformer("jamescalam/minilm-arxiv-encoder")

<h1>Obtener las clases y propiedades con grobid</h1>

In [1]:
from src.processor import PaperProcessor

input_path = "../res/datasets/raw/"
output_path = "../res/datasets/grobid/"

processor = PaperProcessor(output_path=output_path)
papers = processor.process_folder(input_path)

In [3]:
papers[29].authors[0].affiliation.name

'Departments of Computer Science and Statistical Sciences'

<h1>Obtener las clases y propiedades desde los xml ya procesados</h1>

In [1]:
from src.processor import PaperProcessor

input_path = "../res/datasets/raw/"
output_path = "../res/datasets/grobid/"

processor = PaperProcessor(output_path=output_path)
papers = processor.process_folder_from_xml(pdf_path=input_path)

In [3]:
papers[29].authors[0].affiliation.name

'College of Electronics and Information Engineering'